# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [1]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [3]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [ ]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图是一种幽默风格的搞笑比较。图中展示了两种不同状态的狗。左边是一只被幻想成拥有非常发达肌肉、类似人类健美运动员身体的柴犬，配文“16岁的我，工作后的我”，表达了年轻时充满活力和梦想的状态。右边的柴犬则显得普通、略显憔悴，配有文字“好累好困、好想摸鱼、看到鸡腿就开心，不属于我、我也有小肚腩、肉肉的很可爱”，描绘了工作后可能感到疲惫、减少锻炼而略显发福的现实状态。

这种对比通常用来幽默地表达人们对于理想与现实之间差距的感慨，展示了许多人对于年轻时的憧


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [28]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 超高难度的数学试卷图

![gdp_data](./images/gpt-4v-math.jpg)

In [30]:
content = query_base64_image_description("./images/gpt-4v-math.jpg")
print(content)

这幅图包含了一些简单的数学题目，分为左右两列，每列有四个算式。左边的算式主要是减法，右边的则是加法和减法混合。每个算式的答案应该是基础的数学运算，适合小学生练习使用。

左边的题目有：
1. 7 - 3
2. 2 + 8
3. 5 - 5
4. 6 + 3

右边的题目有：
1. 2 + 7
2. 8 - 4
3. 1 + 9
4. 1 - 1

整体上，这是一种帮助学生练习基础数学运算的形式，左侧是减法，右侧是加法，格式整齐，非常适合学生进行数学基础知识的学习和训练。


#### 使用 GPT-4V 识别手写体笔记
![](./images/gpt-4v-1.jpg)

In [32]:
content = query_base64_image_description("./images/gpt-4v.jpg")
print(content)

这张手写笔记的内容主要涉及物理学中与运动相关的理论和公式。具体内容包括：

1. **运动的描述**：
   - 描述了运动的基本概念，如物体的大小和形状、理想化的物体模型等。
   - 引用了物体受力和运动状态的相关情况。

2. **物体的运动分析**：
   - 讨论了物体在空间中的运动以及力的作用。
   - 提到在多个点之间的相对位置和运动情况。

3. **运动方程和公式**：
   - 提供了相关的力和加速度的方程，如 UAC = UAB + UBC 等。
   - 描述了在不同条件下，物体的加速度和力之间的关系。

4. **速度与加速度**：
   - 定义了速度的计算方式以及不同的运动状态。

整体来看，这段内容可能是关于经典力学的学习资料，涵盖了运动学、动力学的一些基本理论和公式。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [33]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张手写笔记的内容主要涉及物理学中与运动相关的理论和公式。具体内容包括：

1. **运动的描述**：
   - 描述了运动的基本概念，如物体的大小和形状、理想化的物体模型等。
   - 引用了物体受力和运动状态的相关情况。

2. **物体的运动分析**：
   - 讨论了物体在空间中的运动以及力的作用。
   - 提到在多个点之间的相对位置和运动情况。

3. **运动方程和公式**：
   - 提供了相关的力和加速度的方程，如 UAC = UAB + UBC 等。
   - 描述了在不同条件下，物体的加速度和力之间的关系。

4. **速度与加速度**：
   - 定义了速度的计算方式以及不同的运动状态。

整体来看，这段内容可能是关于经典力学的学习资料，涵盖了运动学、动力学的一些基本理论和公式。

![](./images/gpt-4v-1.jpg)

In [34]:
content = query_base64_image_description("./images/gpt-4v.jpg")
display(Markdown(content))

这段内容主要讨论物理中关于运动的描述，包括以下几个方面：

1. **运动的基本概念**：
   - 运动体的特性，如大小、形状和物体的全局特性。
   - 描述运动的相关点，涉及质点和物体的相对位置。

2. **运动状态的理解**：
   - 运动体的位置、速度及其对外部环境的影响，如不同物体之间的相互作用。

3. **不同类型的运动分析**：
   - 可能存在的三种情况，探讨在不同条件下运动体的表现。

4. **速度与加速度的定义**：
   - 附件提到速度和加速度的计算公式和物理意义，速度的计算涉及单位时间内的位移。

该笔记总结了运动的基本理论和公式，适合用于物理学习和复习。

In [35]:
def query_base64_image_description_with_markdown(image_path):
   content = query_base64_image_description(image_path)
   content_markdown = Markdown(content)
   return content_markdown

In [38]:
content = query_base64_image_description_with_markdown("./images/gpt-4v-1.jpg")
display(content)

这张图片包含了关于物理学中直线运动的笔记，主要内容包括以下几个方面：

1. **运动的描述**：
   - 运动的特性，如有大小和形状、物体的全方位分布等。
   - 运动点的定义，强调了运动的相对性和参考系。

2. **相对性原则**：
   - 运动体的测速和距离的研究，涉及不同物体之间的相对运动。
   - 运动的描述可能会因参考系的选择而有所不同。

3. **运动方程**：
   - 引入了运动学中的各种基本方程，描述了位置、速度、加速度之间的关系。
   - 提到了关于加速度和位置之间的方程。

4. **速度与加速度**：
   - 速度的定义以及如何表示某一段时间内物体的运动情况。
   - 加速度的概念及其与速度的关系，可能涉及到速度的变化率等。

总体来说，这些内容主要集中在运动的基本概念和物理定律的应用，特别是在直线运动的背景下。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。